Import modules

In [1]:
import numpy as np
import pandas as pd
import os
import zipfile

Extract files from zip

In [2]:
extracted_files = '/Users/himanshuchuri/Desktop/DataScience/Cancer Analysis/CI5plus_Summary_April2019/'
if not os.path.exists(extracted_files): os.mkdir(extracted_files)

In [3]:
path_to_zip_file = '/Users/himanshuchuri/Desktop/DataScience/Cancer Analysis/CI5plus_Summary_April2019.zip'
files_in_zip = ['Americas_Cases.csv','Americas.csv']
with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall(extracted_files)

Load the Data Frames

In [4]:
df_america_cases = pd.read_csv(extracted_files+files_in_zip[0])

In [5]:
df_america_cases.head()

,REGISTRY,YEAR,SEX,CANCER,TOTAL,N0_4,N5_9,N10_14,N15_19,N20_24,...,N45_49,N50_54,N55_59,N60_64,N65_69,N70_74,N75_79,N80_84,N85+,N_UNK
0,7600200,1993,1,1,490,9,2,5,8,6,...,29,39,54,67,54,64,40,22,18,1
1,7600200,1993,1,2,46,0,0,0,1,0,...,6,6,8,7,3,2,2,1,0,0
2,7600200,1993,1,3,19,0,0,0,0,0,...,2,1,3,2,4,2,2,0,1,0
3,7600200,1993,1,4,43,0,0,0,0,0,...,2,3,4,9,4,6,5,3,2,0
4,7600200,1993,1,5,12,0,0,0,0,0,...,0,0,1,3,1,2,0,2,1,0


In [6]:
df_american_regions = pd.read_csv(extracted_files+files_in_zip[1])

In [7]:
df_american_regions.head()

,CONTINENT,AREA,REGISTRY,NAME,YEAR1,YEAR2
0,2,99,7600200,Brazil_Goiania,1993,2012
1,2,99,15200100,Chile_Valdivia,1998,2012
2,2,99,17000100,Colombia_Cali,1983,2012
3,2,99,18800000,Costa_Rica,1982,2011
4,2,99,21800100,Ecuador_Quito,1985,2012


List all of the redundant regions

In [8]:
exclude_values_regions = ['USA_California_Los_Angeles',
'USA_California_Los_Angeles-White','USA_California_Los_Angeles-Hispanic-White',
'USA_California_Los_Angeles-Non-Hispanic-White',
'USA_California_Los_Angeles-Black',
'USA_California_Los_Angeles-Chinese',
'USA_California_Los_Angeles-Japanese',
'USA_California_Los_Angeles-Filipino',
'USA_California_Los_Angeles-Korean',
'USA_SEER-White',
'USA_SEER-Black',
'USA_NPCR',
'USA_NPCR-White',
'USA_NPCR-Black',
'USA_NPCR-American-Indian-Alaskan-Native',
'USA_NPCR-Asian-Pacific-Islander']

In [9]:
df_america_cases['REGION_NAME'] = df_america_cases.apply(lambda row: df_american_regions[(df_american_regions['REGISTRY'] == row['REGISTRY']) & 
                                                        (df_american_regions['YEAR1'] <= row['YEAR']) & 
                                                        (df_american_regions['YEAR2'] >= row['YEAR']) &
                                                        (~df_american_regions['NAME'].isin(exclude_values_regions))].iloc[0]['NAME'] 
                                       if len(df_american_regions[(df_american_regions['REGISTRY'] == row['REGISTRY']) & 
                                                       (df_american_regions['YEAR1'] <= row['YEAR']) & 
                                                       (df_american_regions['YEAR2'] >= row['YEAR']) &
                                                       (~df_american_regions['NAME'].isin(exclude_values_regions))]) > 0 else None, axis=1)

In [10]:
def fix_country_names(text):
  if text == "Costa_Rica":
    return "Costa Rica"
  elif text == "France_Martinique":
    return "Martinique"
  else:
    return text.split("_")[0]


df_america_cases.dropna(inplace=True)
df_america_cases.drop(columns=['REGISTRY', 'TOTAL'],inplace=True, axis=0)
df_america_cases['REGION_NAME'] = df_america_cases['REGION_NAME'].apply(fix_country_names)


In [11]:
df_america_cases = df_america_cases.melt(id_vars=["YEAR", "SEX","CANCER",'REGION_NAME'], 
        var_name="AGE_GROUP", 
        value_name="NUMBER_OF_CASES")

In [12]:
sex_map = {1: 'Male', 2: 'Female'}
cancer_map = {1:'All cancers but non-melanoma skin2 (C00-96, but C44)',
2: 'Lip, oral cavity and pharynx (C00-14)',
3:'Oesophagus (C15)',
4:'Stomach (C16)',
5:'Colon (C18)',
6:'Rectum and anus (C19-21)',
7:'Liver and intrahepatic bile ducts (C22)',
8:'Gallbladder and extrahepatic ducts (C23-24)',
9:'Pancreas (C25)',
10:'Larynx (C32)',
11:'Trachea, bronchus and lung (C33-34)',
12:'Bone (C40-41)',
13:'Melanoma of skin (C43)',
14:'Connective and soft tissue (C47+C49)',
15:'Breast1 (C50)',
16:'Cervix uteri (C53)',
17:'Corpus uteri (C54)',
18:'Ovary (C56)',
19:'Prostate (C61)',
20:'Testis (C62)',
21:'Kidney and renal pelvis (C64-65)',
22:'Bladder (C67)',
23:'Eye (C69)',
24:'Brain, central nervous system (C70-72)',
25:'Thyroid (C73)',
26:'Hodgkin lymphoma (C81)',
27:'Non-Hodgkin lymphoma2 (C82-86, C96)',
28:'Multiple myeloma and immunoproliferative diseases (C88+C90)',
29:'Leukaemia (C91-95)'
}

age_group_map = {'N0_4':'Infants', 'N5_9':'Children', 'N10_14':'Early Teens', 'N15_19': 'Teens', 'N20_24': 'Young Adults (20-24)',
       'N25_29': 'Young Adults (25-29)', 'N30_34': 'Young Adults (30-34)', 'N35_39': 'Adults (30-34)', 'N40_44': 'Adults (40-44)', 'N45_49':'Adults (45-49)', 'N50_54':'Adults (50-54)', 'N55_59':'Adults (55-59)',
       'N60_64': 'Early Seniors (60-64)', 'N65_69': 'Early Seniors (65-69)', 'N70_74':'Seniors (70-74)', 'N75_79':'Seniors (75-79)', 'N80_84':'Elderly (80-84)', 'N85+': 'Elderly (85+)', 'N_UNK':'Unknown'}

In [13]:
df_america_cases['SEX'] = df_america_cases['SEX'].replace(sex_map)

df_america_cases['CANCER'] = df_america_cases['CANCER'].replace(cancer_map)

df_america_cases['AGE_GROUP'] = df_america_cases['AGE_GROUP'].replace(age_group_map)

In [14]:
df_america_cases.to_csv("/Users/himanshuchuri/Desktop/DataScience/Cancer Analysis/cancer_US_with_All_cancers.csv")

In [15]:
df_filtered = df_america_cases[df_america_cases['CANCER'] != 1]  # Filter out rows with cancer=1
df_america_cases.to_csv("/Users/himanshuchuri/Desktop/DataScience/Cancer Analysis/cancer_US_without_All_cancers.csv")
